#Agent in langchain (guy)

LLM -> brain langchain agent

Tools  -> interacts with stock market index thru an api
query against a database
methods/classes
langchain provides some tools
create your own tools as well


LLM  -> ask it a question -> another question

agent is a way to run your llm in a loop in order to complete a task

agenttype
LLM
Tools

In [ ]:
from langchain_core.tools import tool #decorator

Decorator:
- special function that add some extra functionality to original function

In [ ]:
@tool
def multiply(first_int: int, second_int: int) -> int:   # type hinting done to hint to other developers
    """Multiply two integers together."""               # keep detailed docstring gets used as a description for LLM. LLM checks the description not name.
                                                        # you can see docstring on help(multiply)
    return first_int * second_int
                                                        # decorated with @tool that makes multiply function a tool

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


In [ ]:
# creating list of tools (toolkit) to pass to agent

tools = [multiply, add, exponentiate]

In [ ]:
print(add.name)

add


In [ ]:
print(add.description)

Add two integers.


In [ ]:
print(add.args)

{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


**Creating prompts**


Prompt1: User to Agent

Promt2 : Agent to LLM

2 variables:

{input} -> user to agent

{agent_scratchpad} -> intermediate outputs

In [ ]:
# Method 1: can write your own prompt

# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages([
#     ("system": '''u are a math..., use tools to ans....   # system message
#         if u dont have the answer say so,
#         Return only the answer
#         Human: what is 1+1?
#         AI: 2'''),

#     MessagesPlaceholder("chat_history"),                  # includes all the chat history
#     ("human", "{input}"),                                 # current question from human
#     MessagesPlaceholder("agent_scratchpad")               # output from the intermediate tools called
# ])

In [ ]:
# Method 2: Langchain prompts available for download

from langchain import hub                                   # repository -> prompts, chains, agents

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


**Set up OpenAI**

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
# !pip install -q langchain_openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

**Create the agent**

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt) # prompt is prompt that agent gives the llm

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) #verbose=True -> show intermediate details on what is hapening

In [ ]:
agent_executor.invoke( {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking \(3\) to the fifth power and multiplying that by the sum of \(12\) and \(3\), then squaring the whole result is \(13,286,025\).

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking \\(3\\) to the fifth power and multiplying that by the sum of \\(12\\) and \\(3\\), then squaring the whole result is \\(13,286,025\\).'}

In [ ]:
agent_executor.invoke( {
        "input": "What is capital of India?"
    })

# tools are called as and when required



> Entering new AgentExecutor chain...
The capital of India is New Delhi.

> Finished chain.


{'input': 'What is capital of India?',
 'output': 'The capital of India is New Delhi.'}

# chat with excel

In [ ]:
# !pip install -q langchain_experimental langchain_openai

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

In [ ]:
agent = create_csv_agent(
    OpenAI(temperature=0), #llm
    "Titanic-Dataset.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True
)

# think step by step, decide on action, execute the action
# RE: reasoning
# ACT: acting

In [ ]:
agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


{'input': 'how many rows are there?', 'output': '891'}

In [ ]:
agent.invoke("how many people have more than 3 siblings?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the SibSp column is greater than 3, and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].count()PassengerId    30
Survived       30
Pclass         30
Name           30
Sex            30
Age            23
SibSp          30
Parch          30
Ticket         30
Fare           30
Cabin           0
Embarked       30
dtype: int6430 rows have been counted, but I need to specify which column to count.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3]['PassengerId'].count()3030 is the final answer.
Final Answer: 30

> Finished chain.


{'input': 'how many people have more than 3 siblings?', 'output': '30'}

In [ ]:
agent.invoke("how many kites are in the sky?")



> Entering new AgentExecutor chain...
Thought: This is not a valid question for this tool.
Final Answer: This is not a valid question for this tool.

> Finished chain.


{'input': 'how many kites are in the sky?',
 'output': 'This is not a valid question for this tool.'}

# chat with pandas dataframe

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent  # calls python agent to run python code and give output
import pandas as pd

In [ ]:
df = pd.read_csv('Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0), #llm
    df,
    verbose=True,
    allow_dangerous_code=True # colab needs this to use experimental agents, not required otherwiae
)


agent.invoke('how many rows are there?')



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


{'input': 'how many rows are there?', 'output': '891'}

In [ ]:
agent.invoke('how many people have more than 3 siblings?')

# code is written by the python agent and EXECUTED to get the ans. Agents do this but llm will not execute the code if asked directly



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the SibSp column is greater than 3, and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].count()PassengerId    30
Survived       30
Pclass         30
Name           30
Sex            30
Age            23
SibSp          30
Parch          30
Ticket         30
Fare           30
Cabin           0
Embarked       30
dtype: int6430 rows have been counted, but I need to specify which column to count.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3]['PassengerId'].count()3030 is the final answer.
Final Answer: 30

> Finished chain.


{'input': 'how many people have more than 3 siblings?', 'output': '30'}

In [ ]:
agent.invoke("how many kites are in the sky?")



> Entering new AgentExecutor chain...
Thought: This is not a valid question for this tool.
Final Answer: This is not a valid question for this tool.

> Finished chain.


{'input': 'how many kites are in the sky?',
 'output': 'This is not a valid question for this tool.'}